In [167]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import curve_fit
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize

In [168]:
path = "datasets/famcs_students.csv"
df = pd.read_csv(path)
N = 23 % 8
col = df.columns[N]
df.drop(columns=["eye_color", "foot_size", "cource", "group", "height", "sport", "retake"], axis=1, inplace=True)

print(df)


                     stream sex   age   ss          interest       os  \
0    Прикладная информатика   М  20.0  Нет        Математика    MacOS   
1    Прикладная информатика   Ж  20.0  Нет  Программирование    MacOS   
2    Прикладная информатика   Ж  19.0  Нет  Программирование    MacOS   
3    Прикладная информатика   Ж  20.0  Нет        Математика    MacOS   
4    Прикладная информатика   М  20.0  Нет        Математика  Windows   
..                      ...  ..   ...  ...               ...      ...   
122             Информатика   М  20.0  Нет  Программирование  Windows   
123             Информатика   Ж  22.0  Нет  Программирование  Windows   
124             Информатика   М  21.0  Нет        Математика  Windows   
125             Информатика   М  20.0  Нет  Программирование  Windows   
126             Информатика   М  20.0  Нет  Программирование  Windows   

    weekend_study bad_sleep glasses work_experience        ai anime  \
0              Да        Да     Нет     Меньше года 

In [169]:
binary_columns = ['ss', 'weekend_study', 'bad_sleep', 'glasses', 'anime', 'literature']

for column in binary_columns:
    df[column] = df[column].map({'Да': 1, 'Нет': 0})

df["hostel"] = df["hostel"].map({'Нет, я из регионов': 1, 'Нет, я из Минска': 0})
df["study_form"] = df["study_form"].map({'Бюджет': 1, 'Платная': 0})
df["social"] = df["social"].map({'Экстраверт': 1, 'Интроверт': 0})
df["sex"] = df["sex"].map({'М': 1, 'Ж': 0})

categorical_columns = ['stream', 'interest', 'work_experience', 'ai']

df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=False, dtype=int)
df_encoded = df_encoded.dropna()
X = df_encoded.drop(columns=[col])
y = df_encoded[col]

print(X.head())
print(y.head())

   sex   age  ss  weekend_study  bad_sleep  glasses  anime  social  miss  \
0    1  20.0   0              1          1        0      0     1.0   3.0   
1    0  20.0   0              0          0        0      0     1.0   5.0   
2    0  19.0   0              1          0        0      0     1.0  10.0   
3    0  20.0   0              1          1        0      0     1.0   3.0   
4    1  20.0   0              1          0        0      0     0.0   1.0   

   study_form  ...  ai_Cursor  ai_DeepSeek  ai_Gemini  ai_Gemini от Google  \
0           1  ...          1            0          0                    0   
1           1  ...          0            0          0                    0   
2           1  ...          0            0          0                    0   
3           1  ...          0            0          0                    0   
4           1  ...          0            1          0                    0   

   ai_Grok  ai_Mistral  ai_Qwen  ai_claude  ai_gpt oss 120b  \
0        0 

In [170]:
X = X.values
y = y.values

encoder = LabelEncoder()
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15/0.85, random_state=1)


In [171]:
print(y_train.shape, y_val.shape, y_test.shape)

(64,) (14,) (14,)


In [172]:
vals, counts = np.unique(y_train, return_counts=True)
most_frequent = vals[np.argmax(counts)]
print(most_frequent)

def predict_freq(X, most_frequent):
    return np.full(X.shape[0], most_frequent)

2


In [173]:
model_bayes = MultinomialNB()
model_bayes.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [174]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from collections import Counter

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))



for k in range(1, 11):
    model = KNN(k=k)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, predictions)
    print(f"k = {k}: Accuracy = {accuracy:.4f}")


k = 1: Accuracy = 0.5714
k = 2: Accuracy = 0.5714
k = 3: Accuracy = 0.7143
k = 4: Accuracy = 0.6429
k = 5: Accuracy = 0.8571
k = 6: Accuracy = 0.8571
k = 7: Accuracy = 0.8571
k = 8: Accuracy = 0.7143
k = 9: Accuracy = 0.7857
k = 10: Accuracy = 0.6429


In [181]:
model_KNN = KNN(k=6)
model_KNN.fit(X_train, y_train)


In [176]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

class LogisticRegressionOvR:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.classifiers = {}
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.classes = np.unique(y)
        for c in self.classes:
            y_binary = (y == c).astype(int)
            classifier = LogisticRegressionBinary(self.learning_rate, self.n_iterations)
            classifier.fit(X, y_binary)
            self.classifiers[c] = classifier

    def predict_proba(self, X):
        probas = np.array([self.classifiers[c].predict_proba(X) for c in self.classes]).T
        probas = probas / probas.sum(axis=1, keepdims=True)
        return probas

    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

class LogisticRegressionBinary:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear_model)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)


model = LogisticRegressionOvR(learning_rate=0.1, n_iterations=2000)
model.fit(X_train, y_train)

y_proba = model.predict_proba(X_val)

In [177]:
def check_model(y_pred, y_test, average='macro'):
    accuracy = accuracy_score(y_test, y_pred)

    precision = precision_score(y_test, y_pred, average=average)
    recall = recall_score(y_test, y_pred, average=average)

    classes = np.unique(y_test)
    y_test_bin = label_binarize(y_test, classes=classes)
    y_pred_bin = label_binarize(y_pred, classes=classes)

    roc_auc = roc_auc_score(y_test_bin, y_pred_bin, multi_class='ovr', average=average)
    print(accuracy, precision, recall, roc_auc)

In [182]:
check_model(predict_freq(X_test, most_frequent), y_test)
check_model(model_bayes.predict(X_test), y_test)
check_model(model_KNN.predict(X_test), y_test)
check_model(model.predict(X_test), y_test)


0.5 0.16666666666666666 0.3333333333333333 0.5
0.42857142857142855 0.1818181818181818 0.2857142857142857 0.4775132275132275
0.5 0.1794871794871795 0.3333333333333333 0.5099206349206349
0.35714285714285715 0.1851851851851852 0.2380952380952381 0.44510582010582017


C:\Users\Dima\oaid\oiad-11-2025\Hodin\laba1\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Dima\oaid\oiad-11-2025\Hodin\laba1\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
